In [ ]:
import os

os.environ['CLEARML_API_ACCESS_KEY'] = os.getenv("CLEARML_API_ACCESS_KEY")
os.environ['CLEARML_API_SECRET_KEY'] = os.getenv("CLEARML_API_SECRET_KEY")
os.environ["CLEARML_WEB_HOST"]  = "https://app.clear.ml/"
os.environ["CLEARML_API_HOST"]  = "https://api.clear.ml"
os.environ["CLEARML_FILES_HOST"] = "https://files.clear.ml"

In [2]:
import os
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data.datasets import register_pascal_voc
from detectron2.engine import DefaultTrainer
from detectron2.utils.logger import setup_logger
from detectron2.evaluation import PascalVOCDetectionEvaluator
from clearml import Task
from detectron2.engine import HookBase
from detectron2.utils.events import get_event_storage
from numbers import Number


import os, copy
import torch
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_train_loader, detection_utils as utils
from detectron2.data import transforms as T
from detectron2.data.datasets import register_pascal_voc
from detectron2.evaluation import PascalVOCDetectionEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
import random
import numpy as np
import torch
os.environ["PYTHONHASHSEED"] = str(42)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
voc_root = "VOCdevkit"
year = 2012

for split in ["train", "val"]:
    register_pascal_voc(
        f"voc_{str(split)}",
        os.path.join(voc_root, f"VOC{str(year)}"),
        split,
        year
    )

In [4]:
cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file(
        "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"
    )
)

cfg.DATASETS.TRAIN = ("voc_train",)
cfg.DATASETS.TEST = ("voc_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.OUTPUT_DIR = "./output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Solver (optimizer) settings
cfg.SOLVER.IMS_PER_BATCH     = 2
cfg.SOLVER.BASE_LR           = 0.01
cfg.SOLVER.WARMUP_FACTOR     = 1.0 / 1000
cfg.SOLVER.WARMUP_ITERS      = 1000
cfg.SOLVER.MAX_ITER          = 90_000
cfg.SOLVER.STEPS             = (60_000, 80_000)
cfg.SOLVER.GAMMA             = 0.1
cfg.SOLVER.CHECKPOINT_PERIOD = 5000
cfg.TEST.EVAL_PERIOD         = 5000
cfg.SOLVER.AMP.ENABLED       = True


cfg.MODEL.ROI_HEADS.NUM_CLASSES           = 20
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE  = 512


cfg.INPUT.RANDOM_FLIP      = "horizontal"
cfg.INPUT.MIN_SIZE_TRAIN   = (480, 512, 544, 576, 608, 640)
cfg.INPUT.MAX_SIZE_TRAIN   = 1333
cfg.INPUT.CROP.ENABLED     = True
cfg.INPUT.CROP.TYPE        = "absolute_range"
cfg.INPUT.CROP.SIZE        = (400, 600)

cfg.SOLVER.CHECKPOINT_PERIOD = 5000  
cfg.TEST.EVAL_PERIOD       = 5000  

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"
)

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        # If you’d like to save VOC XML/JSON results somewhere:
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        os.makedirs(output_folder, exist_ok=True)
        return PascalVOCDetectionEvaluator(dataset_name)

    # @classmethod
    # def build_train_loader(cls, cfg):
    #     return build_detection_train_loader(cfg, mapper=custom_mapper)


trainer = Trainer(cfg)

[05/03 00:37:33 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/detectron2/engine/train_loop.py:474: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  grad_scaler = GradScaler()


In [ ]:
from detectron2.engine.hooks import BestCheckpointer

experiment_tags = [
    "model_name:faster-rcnn",
    "dataset:voc2012",
    "platform:azureml-notebook",
    "author:hussain",
    "account:hussainsyed.dev@outlook.com",
    "studio:object-detect"
]

task = Task.init(
    project_name="CMT318-Object-Detection",
    task_name="FasterRCNN_R101_Training",
    tags=experiment_tags
)

task.connect(cfg)


class ClearMLHook(HookBase):
    def __init__(self):
        self.logger = Task.current_task().get_logger()
    def after_step(self):
        storage = get_event_storage()
        for name, value in storage.latest().items():
            if isinstance(value, Number):
                self.logger.report_scalar(name, "train", iteration=storage.iter, value=float(value))
            # self.logger.report_scalar(title=name, series="train", iteration=storage.iter, value=value)

trainer.register_hooks([
    BestCheckpointer(
    eval_period=cfg.TEST.EVAL_PERIOD,
    checkpointer=trainer.checkpointer,
    val_metric="bbox/AP50",
    mode="max",
    file_prefix="model_best"
    ),
    ClearMLHook(),
])
trainer.resume_or_load(resume=False)

ClearML Task: overwriting (reusing) task id=ffaf8c441c02417aac9638a716e4a1f4
2025-05-03 00:39:36,287 - clearml.Repository Detection - WARNING - Failed accessing the jupyter server(s): []
2025-05-03 00:39:36,290 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/31ab205b5fdb489d9ad1b4ed44a65563/experiments/ffaf8c441c02417aac9638a716e4a1f4/output/log
[05/03 00:39:37 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl ...


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (21, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (21,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (80, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (80,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias

In [8]:
trainer.train()

[05/03 00:39:43 d2.engine.train_loop]: Starting training from iteration 0
[05/03 00:39:59 d2.utils.events]:  eta: 6:38:54  iter: 59  total_loss: 0.6111  loss_cls: 0.313  loss_box_reg: 0.2443  loss_rpn_cls: 0.02145  loss_rpn_loc: 0.007002    time: 0.2409  last_time: 0.2662  data_time: 0.0046  last_data_time: 0.0035   lr: 0.00059941  max_mem: 1896M
[05/03 00:40:23 d2.utils.events]:  eta: 4:34:01  iter: 179  total_loss: 0.4585  loss_cls: 0.2014  loss_box_reg: 0.2255  loss_rpn_cls: 0.03435  loss_rpn_loc: 0.005361    time: 0.2117  last_time: 0.3012  data_time: 0.0034  last_data_time: 0.0036   lr: 0.0017982  max_mem: 2012M
[05/03 00:40:35 d2.utils.events]:  eta: 4:29:00  iter: 239  total_loss: 0.3408  loss_cls: 0.1605  loss_box_reg: 0.1734  loss_rpn_cls: 0.009077  loss_rpn_loc: 0.006061    time: 0.2048  last_time: 0.2051  data_time: 0.0033  last_data_time: 0.0033   lr: 0.0023976  max_mem: 2012M
[05/03 00:40:42 d2.utils.events]:  eta: 4:27:31  iter: 279  total_loss: 0.448  loss_cls: 0.1784  l

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/detectron2/engine/train_loop.py:493: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.

In [ ]:
from detectron2.evaluation import PascalVOCDetectionEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = PascalVOCDetectionEvaluator("voc_val")
val_loader = build_detection_test_loader(cfg, "voc_val")
final_metrics = inference_on_dataset(trainer.model, val_loader, evaluator)
print("Final VOC2012 val results:", final_metrics)

In [ ]:
output_folder = "./output"

for root, _, files in os.walk(output_folder):
    for fname in files:
        file_path = os.path.join(root, fname)
        # Use a relative path under the output folder as the artifact name
        artifact_name = os.path.relpath(file_path, output_folder)
        task.upload_artifact(name=artifact_name, artifact_object=file_path)
        print(f"Uploaded {artifact_name}")




In [ ]:
task.close()